## Import

This code is modified from a tutorial made by Stacey Shield: https://github.com/UCTMechatronics/pyomo_tutorials

Here we import all the required libraries to symbolically generate the EOM as well as print stuff to the screen
We also reset the console, getting rid of previous data

In [ ]:
%reset
# import libraries
import csv
import sympy as sym
import numpy as np
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition
import datetime

# rerun from here if you don't want to calculate the EOM's again
if 'm' in globals():
    del m # deletes the model
m = ConcreteModel()
N = 30               ########was 30
hm=1/N
m.N = RangeSet(N) 
#m.damping=20

mass_b=0.3   #was 2
mass_l1=1.5 # was 0.5
mass_l2=0.5  #was 0.5

len_b=0.08 #must be 0

len_l1=0.12
len_l2=0.23


m.t_max=5.69 #58kg.cm = 5.69N.m ..... was 12
m.w_max=6.16 #0.17sec/60degree = 352.941176 degree/sec = 6.15998558rad/sec was 44






sym.init_printing()
from IPython.display import display

degr = 3.1415926535/180
def deg2rad(d):
    return d/180*3.1415926535
#print(deg2rad(180),deg2rad(360),deg2rad(60),deg2rad(45),deg2rad(30),deg2rad(15))
#print(180*degr)
print('Done')

## EoM generation
Here we create our symbolic variables for the EoM of the system.

The symbolic variables include the mass, length and inertia of each of the robot links
the generalized coordinates include: X, Z, th_body, th_leg and the leg length (a prismatic leg). Relative angles are used
The generalized coordinates also include the velocities and accelerations. 

q=Generalized coordinates
dq=velocity of the generalized coordinates
ddq=acceleration of the generalized coordinates

Forces are also defined. This robot has a hip torque, tau, and a prismatic force to lengthen and shorten the leg, F
F is the total force, it contains the actuator force as well as a rebound force (hard-stop)

There is also a vertical and horizontal GRF

In [ ]:
# create symbolic variables
g = sym.symbols('g')
mb,ml1, ml2 = sym.symbols(['m_{body}','m_{leg1}', 'm_{leg2}']) # mass

lb = sym.symbols('l_{body}') # length
l1 = sym.symbols('l_{leg1}') # length
l2 = sym.symbols('l_{leg2}') # length
Inb,Inl1,Inl2 = sym.symbols(['I_{body}','I_{leg1}','I_{leg2}']) # moment of intertia

# generalized coordinates
z,thl1,thl2 = sym.symbols(['z','\\theta_{leg1}','\\theta_{leg2}'])
dz,dthl1,dthl2 = sym.symbols(['\dot{z}','\dot{\\theta}_{leg1}','\dot{\\theta}_{leg2}']) 
ddz,ddthl1,ddthl2 = sym.symbols(['\ddot{z}','\ddot{\\theta}_{leg1}','\ddot{\\theta}_{leg2}'])

q = sym.Matrix([[z],[thl1],[thl2]])  # thb removed on September 10 as instructed in order to try correct infeasability issue
dq = sym.Matrix([[dz],[dthl1],[dthl2]])
ddq = sym.Matrix([[ddz],[ddthl1],[ddthl2]])

# forces
#Need to replace F with a torque at the knee and a torque at the hip
tau_hip,tau_knee,GRFx,GRFz = sym.symbols(['\\tau_hip','\\tau_knee','G_x','G_z']) 
print('Done')

The position of the COM of each link. [x,z] is the COM of the body. Assuming the CoM is half way, with uniform inertia
rb is the CoM of the body
rl is the CoM of the leg

the body is horizontal, so the leg attaches to the CoM of the body

In [ ]:

# STEP 1: position vector (COM of each link), as well as other required positions. 
rb = sym.Matrix([[0],
                [z]])

rl = sym.Matrix([[0 + (0.5*l1)*sym.cos(thl1)],
                 [z - (0.5*l1)*sym.sin(thl1)]])

## lower leg
rl2 = sym.Matrix([[0 + l1*sym.cos(thl1) + (0.5*l2)*sym.cos(thl2)],
                  [z - l1*sym.sin(thl1) - (0.5*l2)*sym.sin(thl2)]])###############    ABSOLUTE ANGLES

footx = sym.Matrix([0 + l1*sym.cos(thl1)+ l2*sym.cos(thl2)])
footz = sym.Matrix([z - l1*sym.sin(thl1)- l2*sym.sin(thl2)])     #################
footz = footz[0].simplify()
foot_pos= sym.Matrix([[footx],[0],[footz]]);

print('Done')

From the positions defined above, calculate the velocities

In [ ]:


# STEP 2: generate expressions for the system space velocities from the jacobians
# the Jacobians
Jb = rb.jacobian(q)
Jl = rl.jacobian(q)
Jl2 = rl2.jacobian(q)

vb = Jb*dq
vl = Jl*dq
vl2 = Jl2*dq

footdx = footx.jacobian(q)*dq #This is symbolic again, so it doesnt change with the collocation
footdx = footdx[0].simplify()
print('Done')

Using the positions and velocities, calculate the kinetic and potential energy. This is used to generate the EoM using 
Lagrange Dynamics

In [ ]:


# STEP 3: generate expressions for the kinetic and potential energy
# mass vectors
Mb = sym.Matrix([[mb,mb]])
Ml = sym.Matrix([[ml1,ml1]])
Ml2 = sym.Matrix([[ml2,ml2]])


Ib=sym.Matrix([[Inb]])
Il=sym.Matrix([[Inl1]])
Il2=sym.Matrix([[Inl2]])

#Wb=sym.Matrix([[dthb]]) #removed on September 10
Wl=sym.Matrix([[dthl1]])
Wl2=sym.Matrix([[dthl2]])

# linear kinetic energy:
T1 = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb) + 0.5*Ml*sym.matrix_multiply_elementwise(vl,vl)+ 0.5*Ml2*sym.matrix_multiply_elementwise(vl2,vl2)
# angular kinetic energy:
T2 = 0.5*Il*sym.matrix_multiply_elementwise(Wl,Wl) + 0.5*Il2*sym.matrix_multiply_elementwise(Wl2,Wl2) #this should be a 3*3 matrix of MOI and a vector of angular vel... w'*MOI*w
#removed the weight of the body as instructed on 10 September

#T2 =  0.5*Ib*sym.matrix_multiply_elementwise(Wb,Wb) + 0.5*Il*sym.matrix_multiply_elementwise(Wl,Wl) + 0.5*Il2*sym.matrix_multiply_elementwise(Wl2,Wl2) #this should be a 3*3 matrix of MOI and a vector of angular vel... w'*MOI*w


T = sym.Matrix([T1[0]])+sym.Matrix([T2[0]])
# potential energy
V = mb*g*rb[1] + ml1*g*rl[1] + ml2*g*rl2[1]       #[1] to access the z-coordinate (height)
print('Done')

Use the manipulator equation to calculate the EoM

EOM=M*ddq+C*dq+G-Q-J.'*GRF;

In [ ]:


M=T.jacobian(dq)
M=M.transpose()
M=M.jacobian(dq)              

C  = sym.zeros(len(q),len(q))        # Coriolis Matrix                                                 
for i in range(len(q)):                                             
    for j in range(len(q)):
        for n in range(len(q)):
            C[i,j] = C[i,j]+ 0.5*(sym.diff(M[i,j],q[n]) + sym.diff(M[i,n],q[j]) - sym.diff(M[j,n],q[i]))*dq[n];

G  = sym.zeros(len(q),1)                                         
for i in range(len(q)):
    G[i] = sym.diff(V,q[i]);       

J=foot_pos.jacobian(q)
GRF_l = J.transpose()*sym.Matrix([[GRFx],[0],[GRFz]])  #GRFx 

Q = sym.zeros(len(q),1)
# Q[0]=0                #X
Q[0]=0                #Z
Q[1]=tau_hip-tau_knee           #thl1
Q[2]=tau_knee          #thl2


           
EOM=M*ddq+C*dq+G-Q-GRF_l

EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()
    
print('Done')

## Lambdify
Here we need to lamdify the instructions. Basically linking symbolic lists to variables as well as function mapping..... It converts the symbolic representation to equations that the optimizer can use.....

In [ ]:


# Lambdify


func_map = {'sin':sin, 'cos':cos} 

sym_list = [g,        # constants               ######symlist and variable list must be the same structure
            mb,ml1,ml2, # link mass
            lb,l1,l2,# link lengths
            Inb,Inl1,Inl2,  #link inertias 
            z,thl1,thl2, #q
            dz,dthl1,dthl2, #dq
            ddz,ddthl1,ddthl2, #ddq
            tau_hip,tau_knee, #tau
            GRFx,GRFz] #grf
            
lambEOM_y = sym.lambdify(sym_list,EOMs[0],modules = [func_map])
#lambEOM_thb = sym.lambdify(sym_list,EOMs[1],modules = [func_map]) removed on September 10th
lambEOM_thl = sym.lambdify(sym_list,EOMs[1],modules = [func_map])
lambEOM_thl2 = sym.lambdify(sym_list,EOMs[2],modules = [func_map])
#lambEOM_x = sym.lambdify(sym_list,EOMs[4],modules = [func_map])


lamb_footz = sym.lambdify(sym_list,footz,modules = [func_map])
lamb_footdx = sym.lambdify(sym_list,footdx,modules = [func_map])
print('Done')

## Trajectory optimization
Here we define all the variables for the trajectory optimization problem (do not confuse them with the variables we defined above for the EoM). All variables defined are linked to the model (m). 

N= node, how many time points the trajectory is split into
cN= collocation points, number of sub points per collocation point

q= generalized coordinates for the node points and collocation points
q0= generalized coordinates for the node points only

The time bounds are similarily split up as well

![Drag Racing](collocation.jpg)


In [ ]:


# Variables
cN = 3
m.cN =RangeSet(cN)

links = [('body',1),('leg1',1),('leg2',1)]
m.L = Set(dimen=2, initialize=links)

DOFs = ['z','theta_l1', 'theta_l2'] # generalized coordinates ##################
m.DOF = Set(initialize = DOFs) 

# system coordinates
m.q = Var(m.N, m.cN, m.DOF) # position
m.dq = Var(m.N, m.cN, m.DOF) # velocity
m.ddq = Var(m.N, m.cN, m.DOF) # acceleration

m.q0 = Var(m.N, m.DOF) # position
m.dq0 = Var(m.N, m.DOF) # velocity

m.tt = Var(m.N, m.cN)
m.tt0 = Var(m.N)


m.h = Var(m.N, bounds =(0.1,2.0))

a = [[0.19681547722366, 0.39442431473909, 0.37640306270047],
     [-0.06553542585020, 0.29207341166523, 0.51248582618842],
     [0.02377097434822, -0.04154875212600, 0.11111111111111]]
print('Done')

Here we define the parameters of the model
g=gravity
pi=pi

all the masses and inertia values (converting from symbol to numerical value)

In [ ]:


# PARAMETERS-----------------------------------------------------------------------------------------------------------------
m.g = Param(initialize = 9.81)
pi=sym.pi

m.mb = Param(initialize = mass_b)
m.ml1 = Param(initialize = mass_l1)
m.ml2 = Param(initialize = mass_l2)
m.lb = Param(initialize = len_b)
m.l1 = Param(initialize = len_l1)
m.l2 = Param(initialize = len_l2)

m.Inb= Param(initialize = m.mb.value*m.lb.value**2/12)   
m.Inl1= Param(initialize = m.ml1.value*m.l1.value**2/12)  ##############################m.ml.value*m.q[m.n,m.cN,'r']**2/12
m.Inl2=Param(initialize = m.ml2.value*m.l2.value**2/12)

# mbody = mass_b+mass_l
mbody = mass_b+mass_l1+mass_l2
BW = mbody*m.g.value

print('Done')

#### Collocation constraints
Here we implement the collocation constraints (see image above as to how q0 and q are linked)

In [ ]:


def ContConstraint_p (m, n, dof):
    if n > 1:
        return m.q0[n,dof] == m.q[n-1,3,dof]
    else:
        return Constraint.Skip
m.ContConstraint_p = Constraint(m.N, m.DOF, rule = ContConstraint_p)
def ContConstraint_v (m, n, dof):
    if n > 1:
        return m.dq0[n,dof] == m.dq[n-1,3,dof]
    else:
        return Constraint.Skip
m.ContConstraint_v = Constraint(m.N, m.DOF, rule = ContConstraint_v)
def ContConstraint_tt (m, n):
    if n > 1:
        return m.tt0[n] == m.tt[n-1,3]
    else:
        return Constraint.Skip
m.ContConstraint_tt = Constraint(m.N, rule = ContConstraint_tt)
print('Done')

#### Integration: 3 point collocation, Runge Kutta integration 
Here we implement the integration

In [ ]:


def integrate_p(m, n, c, dof): # for positions
    if n > 0:
        return m.q[n,c,dof] == m.q0[n, dof] + hm*m.h[n]*(a[0][c-1]*m.dq[n,1,dof] \
                                                         +a[1][c-1]*m.dq[n,2,dof] \
                                                         +a[2][c-1]*m.dq[n,3,dof])
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to
m.integrate_p = Constraint(m.N, m.cN, m.DOF, rule = integrate_p)
def integrate_v(m,n,c,dof): # for velocities
    if n > 0:
        return m.dq[n,c,dof] == m.dq0[n, dof] + hm*m.h[n]*(a[0][c-1]*m.ddq[n,1,dof] \
                                                           +a[1][c-1]*m.ddq[n,2,dof] \
                                                           +a[2][c-1]*m.ddq[n,3,dof]) #hm split between different collocation points
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.cN, m.DOF, rule = integrate_v)
def integrate_tt(m,n,c): # for time 
    if n > 0:
        return m.tt[n,c] == m.tt0[n] + hm*m.h[n]*(a[0][c-1]+a[1][c-1]+a[2][c-1]) #log tt0 if just for node points and tt if collocation points because tt is time between node and collocation points
    else:
        return Constraint.Skip 
m.integrate_tt = Constraint(m.N, m.cN, rule = integrate_tt)
print('Done')

#### GRF stuff

Here we set up the + and - sign of the GRF. See: A direct method for trajectory optimization of rigid bodies through contact
https://journals.sagepub.com/doi/abs/10.1177/0278364913506757
OR
https://groups.csail.mit.edu/robotics-center/public_papers/Posa13.pdf
 
Look at equations 8 to 16. Eq 8 and 9 force positive variables. Eq 10, 11 and 12 are technically not needed. 
 
Eq 13 is handled as follows:
m.a_contact_1[n,c] == lamb_footz(*var_list)
m.b_contact_1[n,c] == m.GRF[n,c,'Z','ps']
a_contact_1_pr = m.a_contact_1[n,1] + m.a_contact_1[n,2] + m.a_contact_1[n,3] 
b_contact_1_pr = m.b_contact_1[n,1] + m.b_contact_1[n,2] + m.b_contact_1[n,3]  
return a_contact_1_pr*b_contact_1_pr <= m.eps
 
Eq 14 is handled as follows:
m.a_friction_1[n,c] == m.mu*m.GRF[n,c,'Z','ps'] - (m.GRF[n,c,'X','ps'] + m.GRF[n,c,'X','ng'])
m.b_friction_1[n,c] == m.gamma_1[n,c]
a_friction_1_pr = m.a_friction_1[n,1] + m.a_friction_1[n,2] + m.a_friction_1[n,3] 
b_friction_1_pr = m.b_friction_1[n,1] + m.b_friction_1[n,2] + m.b_friction_1[n,3]  
return a_friction_1_pr*b_friction_1_pr <= m.eps
 
Eq 15 is handled as follows:
m.a_slip_p_1[n,c] ==m.gamma_1[n,c]+lamb_footdx(*var_list)
m.b_slip_p_1[n,c] == m.GRF[n,c,'X','ps']
a_slip_p_1_pr = m.a_slip_p_1[n,1] + m.a_slip_p_1[n,2] + m.a_slip_p_1[n,3] 
b_slip_p_1_pr = m.b_slip_p_1[n,1] + m.b_slip_p_1[n,2] + m.b_slip_p_1[n,3]  
return a_slip_p_1_pr*b_slip_p_1_pr <= m.eps
 
Eq 16 is handled as follows: 
m.a_slip_n_1[n,c] ==m.gamma_1[n,c]-lamb_footdx(*var_list)
m.b_slip_n_1[n,c] == m.GRF[n,c,'X','ng']
a_slip_n_1_pr = m.a_slip_n_1[n,1] + m.a_slip_n_1[n,2] + m.a_slip_n_1[n,3] 
b_slip_n_1_pr = m.b_slip_n_1[n,1] + m.b_slip_n_1[n,2] + m.b_slip_n_1[n,3]  
return a_slip_n_1_pr*b_slip_n_1_pr <= m.eps



In [ ]:


signs = ['ps', 'ng']
m.sgn = Set(initialize = ['ps', 'ng'])
WDOFs = ['X','Z','THETA1']
m.WDOF = Set(initialize=WDOFs) #change the coordinates
m.GRF = Var(m.N, m.cN, m.WDOF, m.sgn, bounds = (0.0,None)) # should bound it to 10*m_robot*g
m.eps = Param(initialize=1000.0, mutable=True)
m.mu = Param(initialize =1.00)
print('Done')

define all the slack variables

In [ ]:


# SLACK VARIABLES
m.a_contact_1  = Var(m.N, m.cN, bounds = (0.0,None)) 
m.b_contact_1  = Var(m.N, m.cN, bounds = (0.0,None)) 
m.a_friction_1 = Var(m.N, m.cN, bounds = (0.0,None)) 
m.b_friction_1 = Var(m.N, m.cN, bounds = (0.0,None)) 
m.a_slip_p_1   = Var(m.N, m.cN, bounds = (0.0,None)) 
m.b_slip_p_1   = Var(m.N, m.cN, bounds = (0.0,None)) 
m.a_slip_n_1   = Var(m.N, m.cN, bounds = (0.0,None)) 
m.b_slip_n_1   = Var(m.N, m.cN, bounds = (0.0,None)) 
m.gamma_1      = Var(m.N, m.cN, bounds = (0.0,None)) 
print('Done')

Handle the contact event

In [ ]:


# CONTACT
def def_a_contact_1(m,n,c):   
        var_list = [m.g,
            m.mb,m.ml1,m.ml2,
            m.lb, m.l1,m.l2,
            m.Inb,m.Inl1,m.Inl2,
            m.q[n,c,'z'],m.q[n,c,'theta_l1'],m.q[n,c,'theta_l2'],
            m.dq[n,c,'z'],m.dq[n,c,'theta_l1'],m.dq[n,c,'theta_l2'],
            m.ddq[n,c,'z'],m.ddq[n,c,'theta_l1'],m.ddq[n,c,'theta_l2'],
            0,0,
            0,0]
        return m.a_contact_1[n,c] == lamb_footz(*var_list)    ########GROUND ADDED HERE!!!!!
m.def_a_contact_1 = Constraint(m.N, m.cN, rule = def_a_contact_1)
def def_b_contact_1(m,n,c):   
    return m.b_contact_1[n,c] == m.GRF[n,c,'Z','ps']
m.def_b_contact_1 = Constraint(m.N, m.cN, rule = def_b_contact_1)
print('Done')

Handle the friction cone

In [ ]:


# FRICTION CONE
def def_a_friction_1(m,n,c):   
    return m.a_friction_1[n,c] == m.mu*m.GRF[n,c,'Z','ps'] - (m.GRF[n,c,'X','ps'] + m.GRF[n,c,'X','ng'])
m.def_a_friction_1 = Constraint(m.N, m.cN, rule = def_a_friction_1)
def def_b_friction_1(m,n,c):   
    return m.b_friction_1[n,c] == m.gamma_1[n,c]
m.def_b_friction_1 = Constraint(m.N, m.cN, rule = def_b_friction_1)
print('Done')

Handle the foot slipping (make sure it slips in the correct direction)

In [ ]:


#SLIP POS DIR
def def_a_slip_p_1(m,n,c):  

    var_list = [m.g,
            m.mb,m.ml1,m.ml2,
            m.lb, m.l1,m.l2,
            m.Inb,m.Inl1,m.Inl2,
            m.q[n,c,'z'],m.q[n,c,'theta_l1'],m.q[n,c,'theta_l2'],
            m.dq[n,c,'z'],m.dq[n,c,'theta_l1'],m.dq[n,c,'theta_l2'],
            m.ddq[n,c,'z'],m.ddq[n,c,'theta_l1'],m.ddq[n,c,'theta_l2'],
            0,0,
            0,0]
    return m.a_slip_p_1[n,c] ==m.gamma_1[n,c]+lamb_footdx(*var_list)
m.def_a_slip_p_1 = Constraint(m.N, m.cN, rule = def_a_slip_p_1)
def def_b_slip_p_1(m,n,c):   
    return m.b_slip_p_1[n,c] == m.GRF[n,c,'X','ps']
m.def_b_slip_p_1 = Constraint(m.N, m.cN, rule = def_b_slip_p_1)



# SLIP NEG DIR
def def_a_slip_n_1(m,n,c):   
    var_list = [m.g,
            m.mb,m.ml1,m.ml2,
            m.lb, m.l1,m.l2,
            m.Inb,m.Inl1,m.Inl2,
            m.q[n,c,'z'],m.q[n,c,'theta_l1'],m.q[n,c,'theta_l2'],
            m.dq[n,c,'z'],m.dq[n,c,'theta_l1'],m.dq[n,c,'theta_l2'],
            m.ddq[n,c,'z'],m.ddq[n,c,'theta_l1'],m.ddq[n,c,'theta_l2'],
            0,0,
            0,0]
    return m.a_slip_n_1[n,c] ==m.gamma_1[n,c]-lamb_footdx(*var_list)
m.def_a_slip_n_1 = Constraint(m.N, m.cN, rule = def_a_slip_n_1)
def def_b_slip_n_1(m,n,c):   
    return m.b_slip_n_1[n,c] == m.GRF[n,c,'X','ng']
m.def_b_slip_n_1 = Constraint(m.N, m.cN, rule = def_b_slip_n_1)

print('Done')

Sum the complimentarity parts across the collocation points and evaluate at the node points

In [ ]:


def def_contact_1_pr(m,n):
    a_contact_1_pr = m.a_contact_1[n,1] + m.a_contact_1[n,2] + m.a_contact_1[n,3] 
    b_contact_1_pr = m.b_contact_1[n,1] + m.b_contact_1[n,2] + m.b_contact_1[n,3]  
    return a_contact_1_pr*b_contact_1_pr <= m.eps
m.def_contact_1_pr = Constraint(m.N, rule = def_contact_1_pr)

def def_friction_1_pr(m,n):
    a_friction_1_pr = m.a_friction_1[n,1] + m.a_friction_1[n,2] + m.a_friction_1[n,3] 
    b_friction_1_pr = m.b_friction_1[n,1] + m.b_friction_1[n,2] + m.b_friction_1[n,3]  
    return a_friction_1_pr*b_friction_1_pr <= m.eps
m.def_friction_1_pr = Constraint(m.N, rule = def_friction_1_pr)

def def_slip_p_1_pr(m,n):
    a_slip_p_1_pr = m.a_slip_p_1[n,1] + m.a_slip_p_1[n,2] + m.a_slip_p_1[n,3] 
    b_slip_p_1_pr = m.b_slip_p_1[n,1] + m.b_slip_p_1[n,2] + m.b_slip_p_1[n,3]  
    return a_slip_p_1_pr*b_slip_p_1_pr <= m.eps
m.def_slip_p_1_pr = Constraint(m.N, rule = def_slip_p_1_pr)

def def_slip_n_1_pr(m,n):
    a_slip_n_1_pr = m.a_slip_n_1[n,1] + m.a_slip_n_1[n,2] + m.a_slip_n_1[n,3] 
    b_slip_n_1_pr = m.b_slip_n_1[n,1] + m.b_slip_n_1[n,2] + m.b_slip_n_1[n,3]  
    return a_slip_n_1_pr*b_slip_n_1_pr <= m.eps
m.def_slip_n_1_pr = Constraint(m.N, rule = def_slip_n_1_pr)
print('Done')

#### Motor model

Here we implement a motor model. In this example we implement it through bounds (see bounds section) and a liner motor model. See: Do Intermediate Gaits Matter When Rapidly Accelerating? https://static1.squarespace.com/static/542ddec8e4b0158794bd1036/t/5e3e0275c0b9914844da5cc0/1581122167965/Fisher_etal_2019.pdf eq(6)

In [ ]:

m.tau_k = Var(m.N, bounds = (-m.t_max,m.t_max)) # actuator torque at knee
m.tau_h = Var(m.N, bounds = (-m.t_max,m.t_max)) # actuator torque at hip

def def_hip_motor_model_p(m,n):
    return  m.tau_h[n] <= m.t_max-m.t_max/m.w_max*(m.dq0[n,'theta_l1'])   #-m.dq0[n,'theta_b'])
#m.def_hip_motor_model_p = Constraint(m.N, rule=def_hip_motor_model_p)
                                                   
                
                                                   
def def_hip_motor_model_n(m,n):
    return  m.tau_h[n] >= -m.t_max-m.t_max/m.w_max*(m.dq0[n,'theta_l1'])   #-m.dq0[n,'theta_b'])
#m.def_hip_motor_model_n = Constraint(m.N, rule = def_hip_motor_model_n)

# def def_hip_motor_model_p(m,n):
# return  m.tau_h[n] <= m.t_max-m.t_max/m.w_max*(m.dq0[n,'theta_l1']-m.dq0[n,'theta_b'])
# m.def_hip_motor_model_p = Constraint(m.N, rule = def_hip_motor_model_p)
# def def_hip_motor_model_n(m,n):
# return  m.tau_h[n] >= -m.t_max-m.t_max/m.w_max*(m.dq0[n,'theta_l1']-m.dq0[n,'theta_b'])
# m.def_hip_motor_model_n = Constraint(m.N, rule = def_hip_motor_model_n)

def def_hip_motor_model_p(m,n):
    return  m.tau_k[n] <= m.t_max-m.t_max/m.w_max*(m.dq0[n,'theta_l2']-m.dq0[n,'theta_l1'])
#m.def_hip_motor_model_p = Constraint(m.N, rule = def_hip_motor_model_p)
                                                    
def def_hip_motor_model_n(m,n):
    return  m.tau_k[n] >= -m.t_max-m.t_max/m.w_max*(m.dq0[n,'theta_l2']-m.dq0[n,'theta_l1'])
#m.def_hip_motor_model_n = Constraint(m.N, rule = def_hip_motor_model_n)

print('Done')

#### EoM constraints
Make sure the EoM are satisfied

In [ ]:
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

S=1.0 #BW/m.mTot    #Scaling factor, I avoid scaling the problem, others seem to think it works....




#for these below you need to have all the symbols, so both torques, not just 1!


print(m.tau_h)

################# Took this out because Q matrix now only consists of 0,1,2,3 and i removed
################# lambEOM_x above



def EOM_y(m,n,c):
    t_hip = S*(m.tau_h[n])
    t_knee = S*(m.tau_k[n])
    Gx_in = S*(m.GRF[n,c,'X','ps']-m.GRF[n,c,'X','ng'])
    Gz_in = S*(m.GRF[n,c,'Z','ps'])

    var_list = [m.g,
            m.mb,m.ml1,m.ml2,
            m.lb, m.l1,m.l2,
            m.Inb,m.Inl1,m.Inl2,
            m.q[n,c,'z'],m.q[n,c,'theta_l1'],m.q[n,c,'theta_l2'],
            m.dq[n,c,'z'],m.dq[n,c,'theta_l1'],m.dq[n,c,'theta_l2'],
            m.ddq[n,c,'z'],m.ddq[n,c,'theta_l1'],m.ddq[n,c,'theta_l2'],
            t_hip,t_knee,
            Gx_in,Gz_in]
    return lambEOM_y(*var_list) == 0
m.EOM_y = Constraint(m.N, m.cN, rule = EOM_y)

def EOM_thl(m,n,c):
    t_hip = S*(m.tau_h[n])
    t_knee = S*(m.tau_k[n])
    Gx_in = S*(m.GRF[n,c,'X','ps']-m.GRF[n,c,'X','ng'])
    Gz_in = S*(m.GRF[n,c,'Z','ps'])
    
    var_list = [m.g,
            m.mb,m.ml1,m.ml2,
            m.lb, m.l1,m.l2,
            m.Inb,m.Inl1,m.Inl2,
            m.q[n,c,'z'],m.q[n,c,'theta_l1'],m.q[n,c,'theta_l2'],
            m.dq[n,c,'z'],m.dq[n,c,'theta_l1'],m.dq[n,c,'theta_l2'],
            m.ddq[n,c,'z'],m.ddq[n,c,'theta_l1'],m.ddq[n,c,'theta_l2'],
            t_hip,t_knee,
            Gx_in,Gz_in]
    return lambEOM_thl(*var_list) == 0
m.EOM_thl = Constraint(m.N, m.cN, rule = EOM_thl)

def EOM_th2(m,n,c):
    t_hip = S*(m.tau_h[n])
    t_knee = S*(m.tau_k[n])
    Gx_in = S*(m.GRF[n,c,'X','ps']-m.GRF[n,c,'X','ng'])
    Gz_in = S*(m.GRF[n,c,'Z','ps'])
    
    var_list = [m.g,
            m.mb,m.ml1,m.ml2,
            m.lb, m.l1,m.l2,
            m.Inb,m.Inl1,m.Inl2,
            m.q[n,c,'z'],m.q[n,c,'theta_l1'],m.q[n,c,'theta_l2'],
            m.dq[n,c,'z'],m.dq[n,c,'theta_l1'],m.dq[n,c,'theta_l2'],
            m.ddq[n,c,'z'],m.ddq[n,c,'theta_l1'],m.ddq[n,c,'theta_l2'],
            t_hip,t_knee,
            Gx_in,Gz_in] 
    return lambEOM_thl2(*var_list) == 0
m.EOM_th2 = Constraint(m.N, m.cN, rule = EOM_th2)







print('Done')
print('Need functions for EOM_r2 and EOM_thl')

#### Bounds
Bound all variables here. Bound to either enforce something (eg leg length) or bound sufficiently high to reduce the search space but not effect the solution. For example if robot runs from 0m to 5m, bound the x variable to -1m to 6m. This does not effect the solution space, but drastically reduces the search space (x was -inf to inf).

In [ ]:

#WHAT ABOUT th2



# def setxBounds(m,n):
#     return (-1.0, m.q0[n,'x'], m.distance+1)
# m.setxBounds = Constraint(m.N, rule=setxBounds)


def setzBounds(m,n):
    return (0.0, m.q0[n,'z'], 3.0)                 #########was 0 and 3
m.setzBounds = Constraint(m.N, rule=setzBounds)

# def setthbBounds(m,n):
#     return (-3.14/2, m.q0[n,'theta_b'], 3.14/2)
# m.setthbBounds = Constraint(m.N, rule=setthbBounds)

def setthl1Bounds(m,n):
    return (30*degr, m.q0[n,'theta_l1'], 90*degr)       ########was -3.14 and 3.14
m.setthl1Bounds = Constraint(m.N, rule=setthl1Bounds)


def setthl2Bounds(m,n):
    return (90*degr, m.q0[n,'theta_l2'], 145*degr)         ########was -3.14 and 3.14
m.setthl2Bounds = Constraint(m.N, rule=setthl2Bounds)


# def setdxBounds(m,n):
#     return (-1, m.dq0[n,'x'],10)   #was 10
# m.setdxBounds = Constraint(m.N, rule=setdxBounds)


def setdzBounds(m,n):
    return (-10, m.dq0[n,'z'],10)
m.setdzBounds = Constraint(m.N, rule=setdzBounds)

# def setdthbBounds(m,n):
#     return (-20, m.dq0[n,'theta_b'], 20)
# m.setdthbBounds = Constraint(m.N, rule=setdthbBounds)

def setdthl1Bounds(m,n):   
    return (-m.w_max, m.dq0[n,'theta_l1'], m.w_max)
m.setdthl1Bounds = Constraint(m.N, rule=setdthl1Bounds)



def setdthl2Bounds(m,n):
    return (-m.w_max, m.dq0[n,'theta_l2'], m.w_max)
m.setdthl2Bounds = Constraint(m.N, rule=setdthl2Bounds)

# def setdrBounds(m,n):
#     return (-m.dl_max, m.dq0[n,'l1'],m.dl_max)
# m.setdrBounds = Constraint(m.N, rule=setdrBounds)


print('Done')

#### Seed generation
In order to search the solution space, a random seed point (start point) is generated. This means that occasionally you will not get a solution, and if you do, it may be a bad solution. In that case, generate a bunch of seeds or start the optimizer with a good seed point

In [ ]:


R = 1.0
D = 5.0

Domain = np.linspace(0,D,N)
Range = np.linspace(0,R,N)
for n in range(1,N+1):
    for c in range (1, cN+1): 
        #Randomizing Q Matrix
        
        
        #m.q[n,c,'x'].value = Domain[n-1] + np.random.uniform(-0.25,0.25)  
        
        
        m.q[n,c,'z'].value  = Range[n-1] + np.random.uniform(-0.25,0.25)
       # m.q[n,c,'theta_b'].value  = np.random.uniform(0.0,np.pi/2) removed on September 10
        m.q[n,c,'theta_l1'].value  = np.random.uniform(0.0,np.pi/2)
        m.q[n,c,'theta_l2'].value  = np.random.uniform(0.0,np.pi/2)
        
        
        #m.q[n,c,'l1'].value  = np.random.uniform(0.0,0.5)
        
        #Setting everything else to 0.01
        for dof in DOFs:
            m.dq[n,c , dof].value = 0.01
            m.ddq[n,c , dof].value = 0.01


        m.GRF[n,c,'Z','ps'].value = 0.01
        m.GRF[n,c,'X','ps'].value = 0.01 
        m.GRF[n,c,'X','ng'].value = 0.01 
        
print('Done')

#### End conditions
In order to enforce a steady-state step, a number of start and end conditions are implemented
generally start conditions are enforced through bounds and end conditions are enforced through constraints.

In [ ]:

# constraints for steady-state 



#  m.q[1,1,'x'].value = 0.0
#  m.q[1,1,'x'].fixed = True
droptest = 1
jumptest = 2

test =2    # Make selection here

if test == droptest:
    m.q[1,1,'z'].value = 0.5       ################# starting height was 2.0 other good value is 0.75 when dq z is 8
    m.q[1,1,'z'].fixed = True     #################

    m.q[1,1,'theta_l1'].value = 90*degr    
    m.q[1,1,'theta_l1'].fixed = True

    m.q[1,1,'theta_l2'].value = 90*degr       
    m.q[1,1,'theta_l2'].fixed = True

    m.dq[1,1,'z'].value = 0.1         ################# starting velocity was 0 other good value is 8 when q z is 0.75
    m.dq[1,1,'z'].fixed = True

if test == jumptest:
    m.q[1,1,'z'].value = len_l1+len_l2  ############### starting height was 2.0 other good value is 0.75 when dq z is 8
    m.q[1,1,'z'].fixed = True           #################

    m.q[1,1,'theta_l1'].value = 60*degr    
    m.q[1,1,'theta_l1'].fixed = True

    m.q[1,1,'theta_l2'].value = 120*degr       
    m.q[1,1,'theta_l2'].fixed = True

    m.dq[1,1,'z'].value = 0.0      ################# starting velocity was 0 other good value is 8 when q z is 0.75
    m.dq[1,1,'z'].fixed = True

# def finalX(m,n):
#     if (n==N) :
#         return m.q0[n,'x'] >= m.distance
#         #return m.q0[n,'x'] == m.q0[1,'x']
#     else:
#         return Constraint.Skip
#m.finalX = Constraint(m.N, rule = finalX)    ####################


def finalZ(m,n):
    if (n==N) :
        return m.q0[n,'z'] >= 0.42      #    m.q0[1,'z']       ############ was return m.q0[n,'z'] >= 0.5 and return m.q0[n,'z'] == m.q0[1,'z'] 
    else:
        return Constraint.Skip
m.finalZ = Constraint(m.N, rule = finalZ)     ###################


# def finalthb(m,n):                                 #####removed on September 10
#     if (n==N) :
#         return m.q0[n,'theta_b'] == m.q0[1,'theta_b']
#     else:
#         return Constraint.Skip
#m.finalthb = Constraint(m.N, rule = finalthb)    ####################


def finalthl1(m,n):
    if (n==N) :
        return m.q0[n,'theta_l1'] == m.q0[1,'theta_l1']
    else:
        return Constraint.Skip
m.finalthl1 = Constraint(m.N, rule = finalthl1)     ######################

#################

def finalthl2(m,n):
    if (n==N) :
        return m.q0[n,'theta_l2'] == m.q0[1,'theta_l2']
    else:
        return Constraint.Skip
m.finalthl2 = Constraint(m.N, rule = finalthl2)       ######################








# def finaldx(m,n):
#     if (n==N) :
#         return m.dq0[n,'x'] == m.dq0[1,'x']
#     else:
#         return Constraint.Skip
# #m.finaldx = Constraint(m.N, rule = finaldx)      #####################




def finaldz(m,n):
    if (n==N) :
        return m.dq0[n,'z'] == m.dq0[1,'z']
    else:
        return Constraint.Skip
m.finaldz = Constraint(m.N, rule = finaldz)      #####################

# def finaldthb(m,n):
#     if (n==N) :
#         return m.dq0[n,'theta_b'] == m.dq0[1,'theta_b']       ###################### removed on September 10
#     else:
#         return Constraint.Skip
# m.finaldthb = Constraint(m.N, rule = finaldthb)      


def finaldthl1(m,n):
    if (n==N) :
        return m.dq0[n,'theta_l1'] == m.dq0[1,'theta_l1']
    else:
        return Constraint.Skip
m.finaldthl1 = Constraint(m.N, rule = finaldthl1)      #####################

###############
def finaldthl2(m,n):
    if (n==N) :
        return m.dq0[n,'theta_l2'] == m.dq0[1,'theta_l2']
    else:
        return Constraint.Skip
m.finaldthl2 = Constraint(m.N, rule = finaldthl2)      ################

# def finaldr(m,n):
#     if (n==N) :
#         return m.dq0[n,'l1'] == m.dq0[1,'l1']
#     else:
#         return Constraint.Skip
# m.finaldr = Constraint(m.N, rule = finaldr)


print('Done')

#### Run the solver

First define the cost. We set the cost to 1, solve the seed, then set the cost to the correct value and re-solve in a loop. 
after each solve, reduce the epsilon value and solve again until within good tolerance

In [ ]:


opt = SolverFactory('ipopt') # standard issue, garden variety ipopt
opt.options["print_level"] = 0 # was 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"] = 100000 # maximum number of iterations
opt.options["max_cpu_time"] = 120000 # maximum cpu time in seconds
opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.


def CostFun(m):
    return 1 
m.Cost = Objective(rule = CostFun)

#solve once with obj=1
results = opt.solve(m, tee=True)
def CostFun(m):
    return sum(((m.tau_k[n]**2+m.tau_h[n]**2)) for n in range(1,N+1)) #Minimum total actuator force and torque
m.Cost = Objective(rule = CostFun)

#loop solve with proper objective
for i in range(0,4):
    print(m.eps.value) ###########
    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        #If the previous solution is optmial, continue solving
        results = opt.solve(m, tee=True)
        m.eps.value = m.eps.value/10
    elif (results.solver.termination_condition == TerminationCondition.infeasible):
        # Break loop when model in infeasible
        break

### Animate the solution








THIS DOES NOT PRINT WRT TIME AND DOES NOT LOOK AT THE COLLOCATION POINTS ONLY THE NODE POINTS 




In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

print(results.solver.status) 
print(results.solver.termination_condition) 

fig1, ax1 = plt.subplots(1,1) #create axes
#ax1.set_aspect('equal')


def plot_robot(i,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-0.75,0.75])    # choose scales according to 2:3 aspect ratio
    ax.set_ylim([-0.1,0.9])
            
    #plot leg 
    leg_xt = 0 
    leg_yt = m.q0[i,'z'].value 
    leg_xb = 0 + m.l1.value*cos(m.q0[i,'theta_l1'].value)
    leg_yb = m.q0[i,'z'].value - m.l1.value*sin(m.q0[i,'theta_l1'].value)
    ax.plot([leg_xt,leg_xb],[leg_yt,leg_yb],color='xkcd:black')
    
    #plot leg2 
    leg2_xt = 0 + m.l1.value*cos(m.q0[i,'theta_l1'].value)
    leg2_yt = m.q0[i,'z'].value - m.l1.value*sin(m.q0[i,'theta_l1'].value)
    leg2_xb = 0 + m.l1.value*cos(m.q0[i,'theta_l1'].value) + m.l2.value*cos(m.q0[i,'theta_l2'].value)
    leg2_yb = m.q0[i,'z'].value - m.l1.value*sin(m.q0[i,'theta_l1'].value) - m.l2.value*sin(m.q0[i,'theta_l2'].value)
    ax.plot([leg2_xt,leg2_xb],[leg2_yt,leg2_yb],color='xkcd:blue')

    #plot ground
    ax.plot([-1.5,1.5],[0,0],color='xkcd:green')

    #plot starting height
    ax.plot([-1.5,1.5],[m.q[1,1,'z'].value,m.q[1,1,'z'].value],color='xkcd:orange')

    
update = lambda i: plot_robot(i,m,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True)
# animate.event_source.start()

HTML(animate.to_jshtml())
# infeasible when bounds are not met

In [ ]:
from numpy import asarray
from numpy import save

print(max(m.tau_h[n].value for n in range (1, N+1)))
print(min(m.tau_h[n].value for n in range (1, N+1)))
print(max(m.tau_k[n].value for n in range (1, N+1)))
print(min(m.tau_k[n].value for n in range (1, N+1)))

#is this going to save the N nodes and 3 collocation points??? 

# dataL2 = asarray([m.l2.value for n in range(1, N+1)])
# print(dataL2)

data = asarray([ #N for n in range (1, N+1)],
#                 [m.lb.value for n in range (1, N+1)],
#                 [m.l1.value for n in range(1, N+1)],
#                 [m.l2.value for n in range(1, N+1)],
#                 [m.tau_h[n].value for n in range (1, N+1)],
#                 [m.tau_k[n].value for n in range (1, N+1)],
                 [m.q0[n,'theta_l1'].value for n in range (1, N+1)],
                 [m.q0[n,'theta_l2'].value for n in range (1, N+1)],
#                 [m.dq[n,1,'z'].value for n in range (1, N+1)],
#                 [m.dq[n,1,'theta_l1'].value for n in range (1, N+1)],
#                 [m.dq[n,1,'theta_l2'].value for n in range (1, N+1)],

                #[m.q0[n,'x'].value for n in range (1, N+1)], #######no x
#                 [m.q0[n,'z'].value for n in range (1, N+1)],
#                 [mass_b for n in range (1, N+1)],
#                 [mass_l1 for n in range (1, N+1)],
#                 [mass_l2 for n in range (1, N+1)],
#                 [len_b for n in range (1, N+1)],
#                 [len_l1 for n in range (1, N+1)],
#                 [len_l2 for n in range (1, N+1)],
#                 [m.q[1,1,'z'].value for n in range (1, N+1)],
                [m.h[n].value for n in range (1, N+1)]])
    
dateStamped = datetime.datetime.now().strftime("output/data%Y%m%d_%H%M%S.npy")
save(dateStamped, data)
filenamefilename = "output/filenames.csv"
np.savetxt(filenamefilename, asarray([dateStamped]), fmt='%s')
'''
filenames = []
with open(filenamefilename) as fp:
    reader = csv.reader(fp, delimiter="\n", quotechar='"')
    for row in reader:
        print(row)
        filenames = [filenames, row[0]]
# filenames = np.load(filenamefilename)
# filenames.append(dateStamped)
np.savetxt(filenamefilename, asarray([filenames]), fmt='%s\n')
'''
print('Done writing to ',dateStamped)